## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from pathlib import Path
from time import time
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.utils import shuffle
from joblib import Parallel, delayed

In [2]:
#DEFININDO ALGUNS PARÂMETROS DO GRÁFICO
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
params = {
    'figure.figsize': [12, 3.3], 
    'axes.labelsize': 12,
    'axes.titlesize':14, 
    'font.size': 12,
    'legend.fontsize': 10, 
    'xtick.labelsize': 10, 
    'ytick.labelsize': 10,
    'axes.axisbelow': True
}
plt.rcParams.update(params)

## Definição de funções

In [3]:
def abrirDataframe(qtdUsuarios, tipo, caminhoPastaCSVs="../../CSV/"):
    if tipo=="treinamento":
        caminhoCSV = list(Path(caminhoPastaCSVs).glob("train_dataset_"+str(qtdUsuarios)+"usuarios*.csv"))[0]
    else:
        caminhoCSV = list(Path(caminhoPastaCSVs).glob("test_dataset_"+str(qtdUsuarios)+"usuarios*.csv"))[0]
    df = pd.read_csv(caminhoCSV)
    return df

In [4]:
def separarXeY(dfTreinamento, dfTeste, featuresSelecionadas=None):
    
    # SELECIONANDO TODAS AS FEATURES SE FEATURESSELECIONADAS VIER NONE
    if featuresSelecionadas == None:
        featuresSelecionadas = df.columns.drop(["potenciaEspiao", "ataquePresente"])
    
    # SEPARANDO OS ARRAYS
    XTrain         = dfTreinamento[featuresSelecionadas]
    XTest          = dfTeste[featuresSelecionadas]
    yTrain         = dfTreinamento["ataquePresente"]
    yTest          = dfTeste["ataquePresente"]
    snrTrain       = dfTreinamento["snr"]
    snrTest        = dfTeste["snr"]
    potEspiaoTrain = dfTreinamento["potenciaEspiao"]
    potEspiaoTest  = dfTeste["potenciaEspiao"]
    
    return XTrain, XTest, yTrain, yTest, snrTrain, snrTest, potEspiaoTrain, potEspiaoTest

In [5]:
def deteccaoMachineLearning(XTrain, XTest, yTrain, objClassificador):

    # TREINANDO O CLASSIFICADOR
    objClassificador.fit(XTrain, yTrain)

    # PREDIZENDO OS DE TESTE
    arrayTempos = []
    yPred       = []
    for i, amostraTeste in XTest.iterrows():
        tic = time()
        yPred.append(objClassificador.predict([amostraTeste.to_numpy()])[0])
        toc = time()
        arrayTempos.append(toc-tic)
        
    return np.array(yPred), np.mean(arrayTempos), np.std(arrayTempos)

In [6]:
def deteccaoHassan(XTest):
    arrayTempos = []
    yPred       = []
    for i, amostraTeste in XTest.iterrows():
        tic = time()
        yPred.append(1 if amostraTeste["E"] > amostraTeste["eta"] else 0)
        toc = time()
        arrayTempos.append(toc-tic)
        
    return np.array(yPred), np.mean(arrayTempos), np.std(arrayTempos)

In [7]:
def deteccaoKapetanovic(XTest, threshold):
    arrayTempos = []
    yPred       = []
    for i, amostraTeste in XTest.iterrows():
        tic = time()
        yPred.append(1 if amostraTeste["a1"]/amostraTeste["a2"] > threshold else 0)
        toc = time()
        arrayTempos.append(toc-tic)
        
    return np.array(yPred), np.mean(arrayTempos), np.std(arrayTempos)

In [8]:
def obterValoresUnicos(arraySNR, arrayPotEspiao):
    rangeSNRs      = np.unique(arraySNR)
    rangePotEspiao = np.unique(arrayPotEspiao)
    return rangeSNRs, rangePotEspiao

In [9]:
def calcularMetricas(yTest, yPred):
    acuracia  = accuracy_score(yTest, yPred)
    precisao  = precision_score(yTest, yPred)
    revocacao = recall_score(yTest, yPred)
    matrizConfusao = confusion_matrix(yTest, yPred)
    
    return acuracia, precisao, revocacao, matrizConfusao

In [10]:
def plotarResultado(matrizProbabilidadeDeteccao, matrizConfusao, rangePotEspiao, rangeSNRs, qtdUsuarios, qtdAntenas, qtdSimbolos, descricao, acuracia, precisao, revocacao):
    
    # MONTANDO O TITULO DA FIGURA
    titulo  = descricao + "\n"
    titulo += "Usuários: " + str(qtdUsuarios) + " - Antenas: " + str(qtdAntenas) + " - Símbolos: " + str(qtdSimbolos) + "\n"
    titulo += "Acurácia: " + str(acuracia)[:7] + " - Precisão: " + str(precisao)[:7] + " - Revocação: " + str(revocacao)[:7] + "\n"
    
    # CRIANDO O PRIMEIRO GRAFICO
    fig, axs = plt.subplots(1, 2)
    for i in range(len(rangePotEspiao)):
        axs[0].plot(rangeSNRs, matrizProbabilidadeDeteccao[i], label="Potência do Espião: "+str(rangePotEspiao[i]))
    axs[0].set_xlabel("SNR")
    axs[0].set_ylabel("Probabilidade de Detecção")
    axs[0].grid(alpha=0.5)
    axs[0].legend()
    
    # CRIANDO A MATRIZ DE CONFUSAO
    axs[1].imshow(matrizConfusao, cmap="gray")
    for (j,i), total in np.ndenumerate(matrizConfusao):
        axs[1].text(i, j, int(total), ha="center", va="center", color="#e6005c", size=15)
    axs[1].set_xlabel("Predito")
    axs[1].set_ylabel("Real")
    axs[1].set_xticklabels([])
    axs[1].set_yticklabels([])
    
    # PRINTANDO
    plt.suptitle(titulo, y=1.15)
    plt.show()

In [11]:
def obterDictNovoResultado(classificador, matrizProbabilidadeDeteccao, matrizConfusao, rangePotEspiao, rangeSNRs, qtdUsuarios, qtdAntenas, qtdSimbolos, featuresSelecionadas, acuracia, precisao, revocacao, tempoMedioPredicao, tempoDesvioPredicao, repetibilidadeDataset):
    
    classificador = classificador.__class__.__name__ if type(classificador) != str else classificador
    descricao = "Deteccao baseada em " + classificador + " com " + " ".join(featuresSelecionadas)
    
    dictNovoResultado = {
        classificador + "_" + str(time()): {
            "qtdUsuarios": int(qtdUsuarios),
            "qtdAntenas": int(qtdAntenas),
            "qtdSimbolos": int(qtdSimbolos),
            "descricao": descricao,
            "repetibilidadeDataset": int(repetibilidadeDataset),
            "acuracia": float(acuracia),
            "precisao": float(precisao),
            "revocacao": float(revocacao),
            "tempoMedioPredicao": tempoMedioPredicao,
            "tempoDesvioPredicao": tempoDesvioPredicao, 
            "featuresSelecionadas": featuresSelecionadas,
            "rangePotEspiao": rangePotEspiao.tolist(),
            "rangeSNRs": rangeSNRs.tolist(),
            "matrizProbabilidadeDeteccao": matrizProbabilidadeDeteccao.tolist(),
            "matrizConfusao": matrizConfusao.tolist()
        }
    }
    
    return dictNovoResultado

In [12]:
def rodarTesteDeteccao(qtdUsuarios, featuresSelecionadas, classificador, esquemaDeteccao, repetibilidadeDataset, thresholdKapetanovic=100):
    
    # PEGO O QUE E O QUE
    XTrain, XTest, yTrain, yTest, snrTrain, snrTest, potEspiaoTrain, potEspiaoTest = separarXeY(abrirDataframe(qtdUsuarios, "treinamento"), abrirDataframe(qtdUsuarios, "teste"), featuresSelecionadas)
    
    # VALORES UNICOS DE SNR E POTENCIA DE ESPIAO PRA EU SABER O INDEX DAS MATRIZES DE RESULTADOS
    rangeSNRs, rangePotEspiao = obterValoresUnicos(snrTest, potEspiaoTest)
    
    # MATRIZES QUE VAO GUARDAR OS RESULTADOS
    matrizProbabilidadeDeteccao = np.zeros(shape=(len(rangePotEspiao), len(rangeSNRs)))
    
    # PREDICAO
    if esquemaDeteccao == "ML":
        yPred, tempoMedioPredicao, tempoDesvioPredicao = deteccaoMachineLearning(XTrain, XTest, yTrain, classificador)
    elif esquemaDeteccao == "Hassan":
        yPred, tempoMedioPredicao, tempoDesvioPredicao = deteccaoHassan(XTest)
    else:
        yPred, tempoMedioPredicao, tempoDesvioPredicao = deteccaoKapetanovic(XTest, thresholdKapetanovic)

    # AGREGO NA MATRIZ DE PROB DE DETECCAO
    for (indexTesteAtual, predicaoAtual) in zip(yTest.index, yPred):

        # PEGANDO EM QUAL LINHA E EM QUAL COLUNA DA MATRIZ EU VOU COLOCAR O RESULTADO
        indexPotEsp = np.where(rangePotEspiao==potEspiaoTest[indexTesteAtual])[0][0]
        indexSNR    = np.where(rangeSNRs==snrTest[indexTesteAtual])[0][0]

        # JA SOMO COM A PROBABILIDADE (DIVIDINDO PELA REPETIBILIDADE)
        matrizProbabilidadeDeteccao[indexPotEsp][indexSNR] += predicaoAtual/repetibilidadeDataset       
    
    # CONSIDERANDO QUE O DATASET ESTA BALANCEADO, HA MUITO MAIS REPETIBILIDADE QUANDO POTESP = 0
    matrizProbabilidadeDeteccao[0] /= (len(rangePotEspiao) - 1)
    
    # METRICAS
    acuracia, precisao, revocacao, matrizConfusao = calcularMetricas(yTest, yPred)
    
    # DICIONARIOS COM OS RESULTADOS 
    dictNovoResultado = obterDictNovoResultado(classificador, matrizProbabilidadeDeteccao, matrizConfusao, rangePotEspiao, rangeSNRs, qtdUsuarios, qtdAntenas, qtdSimbolos, featuresSelecionadas, acuracia, precisao, revocacao, tempoMedioPredicao, tempoDesvioPredicao, repetibilidadeDataset)
    
    return dictNovoResultado

In [13]:
def salvarNovosResultados(arrayResultados, arquivoSalvar):
    for dictNovoResultado in arrayResultados:
        keyNova    = list(dictNovoResultado.keys())[0]
        valuesNovo = dictNovoResultado[list(dictNovoResultado.keys())[0]]
        with open(arquivoSalvar, mode="r+") as file:
            dictResultados = json.load(file)
            dictResultados[keyNova] = valuesNovo
            file.seek(0)
            json.dump(dictResultados, file, indent=4)
            file.truncate()

## Parâmetros iniciais

In [14]:
classificadores       = [DecisionTreeClassifier(), Perceptron(), LinearDiscriminantAnalysis(), SVC()]
rangeQtdUsuarios      = np.array([1, 2, 4, 8, 16, 32, 64])
qtdAntenas            = 256
qtdSimbolos           = 128
repetibilidadeDataset = 1000
nucleosProcessador    = -1
thresholdKapetanovic  = 100
plotar                = True
verbose               = 100
combinacoesFeaturesML = [
    ["E"],
    ["E", "a1", "a2"],
    ["a1", "a2"]
]
dictResultados = {}
arquivoSalvar = Path("../../Resultados/probabilidadeDeteccao_" + str(time()) + ".json")
with open(arquivoSalvar, mode="w") as file:
    json.dump(dictResultados, file)

## Rodando os testes de detecção com machine learning, hassan e kapetanovic

In [15]:
esquemaDeteccao = "ML"
arrayResultados = Parallel(verbose=verbose, n_jobs=nucleosProcessador)(delayed(rodarTesteDeteccao)(qtdUsuarios, featuresSelecionadas, classificador, esquemaDeteccao, repetibilidadeDataset, thresholdKapetanovic) for classificador in classificadores for qtdUsuarios in rangeQtdUsuarios for featuresSelecionadas in combinacoesFeaturesML)
salvarNovosResultados(arrayResultados, arquivoSalvar)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  8

In [16]:
esquemaDeteccao      = "Hassan"
classificador        = esquemaDeteccao
featuresSelecionadas = ["E", "eta"]
arrayResultados      = Parallel(verbose=verbose, n_jobs=nucleosProcessador)(delayed(rodarTesteDeteccao)(qtdUsuarios, featuresSelecionadas, classificador, esquemaDeteccao, repetibilidadeDataset, thresholdKapetanovic) for qtdUsuarios in rangeQtdUsuarios)
salvarNovosResultados(arrayResultados, arquivoSalvar)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done   2 out of   7 | elapsed:  2.1min remaining:  5.4min
[Parallel(n_jobs=-1)]: Done   3 out of   7 | elapsed:  2.2min remaining:  2.9min
[Parallel(n_jobs=-1)]: Done   4 out of   7 | elapsed:  2.2min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done   5 out of   7 | elapsed:  2.2min remaining:   52.8s
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:  2.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:  2.2min finished


In [17]:
esquemaDeteccao      = "Kapetanovic"
classificador        = esquemaDeteccao
featuresSelecionadas = ["a1", "a2"]
arrayResultados      = Parallel(verbose=verbose, n_jobs=nucleosProcessador)(delayed(rodarTesteDeteccao)(qtdUsuarios, featuresSelecionadas, classificador, esquemaDeteccao, repetibilidadeDataset, thresholdKapetanovic) for qtdUsuarios in rangeQtdUsuarios)
salvarNovosResultados(arrayResultados, arquivoSalvar)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done   2 out of   7 | elapsed:  2.2min remaining:  5.6min
[Parallel(n_jobs=-1)]: Done   3 out of   7 | elapsed:  2.2min remaining:  3.0min
[Parallel(n_jobs=-1)]: Done   4 out of   7 | elapsed:  2.2min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done   5 out of   7 | elapsed:  2.2min remaining:   53.7s
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:  2.3min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:  2.3min finished
